In [3]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms

from models import *


global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
    
batch_size = 256
model_name = "VGG16_quant_project_part2_90_prec"   #"Resnet20_quant"
model = VGG16_quant_project_part2()

print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [150, 225]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1           

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
VGG_quant(
  (features): Sequential(
    (0): QuantConv2d(
      3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): QuantConv2d(
      64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): QuantConv2d(
      64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): QuantConv2d(
      128, 128, kernel_size=(3, 3), stride

In [ ]:
lr = 1e-3
weight_decay = 1e-6
epochs = 1000
best_prec = 0
# momentum = 0.9
#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss(label_smoothing=0.1).cuda()
# optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
#cudnn.benchmark = True

from torch.optim.lr_scheduler import CosineAnnealingLR
scheduler = CosineAnnealingLR(
    optimizer,
    T_max=epochs,  # 500
    eta_min=1e-10,  # final LR
)



if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)
        

for epoch in range(0, epochs):

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)
    scheduler.step()

In [4]:
fdir = 'result/'+str(model_name)+'/model_best.pth.tar'

checkpoint = torch.load(fdir)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 9010/10000 (90%)



In [86]:
# Pre-hook to save inputs
class SaveOutput:
    def __init__(self):
        self.outputs = []   # list of (name, module_in) for pre-hooks
    def clear(self):
        self.outputs = []

save_output = SaveOutput()
hook_map = []   # keeps the module name for each saved output

def make_pre_hook(name):
    def hook(module, module_in, module_out=None):
        # store (module_name, module_in tensor)
        save_output.outputs.append((name, module_in))
        hook_map.append(name)
    return hook

# register named pre-hooks only for relevant layer types
for name, module in model.named_modules():
    if isinstance(module, (torch.nn.Conv2d, torch.nn.MaxPool2d, torch.nn.ReLU)):
        module.register_forward_pre_hook(make_pre_hook(name))

# run a single batch to populate save_output
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
save_output.clear()
images, labels = next(iter(testloader))
images = images.to(device)
_ = model(images)

# print mapping of saved outputs
print("Saved-hook index -> module name (in order):")
for idx, nm in enumerate(hook_map):
    print(idx, ":", nm)

# show modules that have weight_q (quantized convs) and find their hook indices
print("\nModules that expose weight_q (quantized conv layers):")
quant_names = []
for name, m in model.named_modules():
    if hasattr(m, 'weight_q'):
        print("  -", name)
        quant_names.append(name)

print("\nHook indices for quantized modules (if present in hook_map):")
for qn in quant_names:
    indices = [i for i, nm in enumerate(hook_map) if nm == qn]
    print(qn, "-> hook indices:", indices)

Saved-hook index -> module name (in order):
0 : conv1
1 : relu
2 : layer1.0.conv1
3 : layer1.0.relu
4 : layer1.0.conv2
5 : layer1.0.relu
6 : layer1.1.conv1
7 : layer1.1.relu
8 : layer1.1.conv2
9 : layer1.1.relu
10 : layer1.2.conv1
11 : layer1.2.relu
12 : layer1.2.conv2
13 : layer1.2.relu
14 : layer2.0.conv1
15 : layer2.0.relu
16 : layer2.0.conv2
17 : layer2.0.downsample.0
18 : layer2.0.relu
19 : layer2.1.conv1
20 : layer2.1.relu
21 : layer2.1.conv2
22 : layer2.1.relu
23 : layer2.2.conv1
24 : layer2.2.relu
25 : layer2.2.conv2
26 : layer2.2.relu
27 : layer3.0.conv1
28 : layer3.0.relu
29 : layer3.0.conv2
30 : layer3.0.downsample.0
31 : layer3.0.relu
32 : layer3.1.conv1
33 : layer3.1.relu
34 : layer3.1.conv2
35 : layer3.1.relu
36 : layer3.2.conv1
37 : layer3.2.relu
38 : layer3.2.conv2
39 : layer3.2.relu

Modules that expose weight_q (quantized conv layers):
  - layer1.0.conv1
  - layer1.0.conv1.weight_quant
  - layer1.0.conv2
  - layer1.0.conv2.weight_quant
  - layer1.1.conv1
  - layer1.1.

In [ ]:
# HW

#  1. Train with 4 bits for both weight and activation to achieve >90% accuracy
#  2. Find x_int and w_int for the 2nd convolution layer
#  3. Check the recovered psum has similar value to the un-quantized original psum
#     (such as example 1 in W3S2)

In [ ]:
#send an input and grap the value by using prehook like HW3


In [87]:
# Find x_int and w_int for the 2nd convolution layer, which is the first quantized layer
mod = dict(model.named_modules())['layer1.0.conv1']
mod.show_params()
w_bit = 4
weight_q = mod.weight_q.detach().cuda() # quantized value is stored during the training
w_alpha = 2.087000 # alpha is defined in model already. bring it out here, should be 8 according to design
w_delta = w_alpha / ((2 ** (w_bit-1))-1)   # delta can be calculated by using alpha and w_bit
weight_int = weight_q / w_delta  # w_int can be calculated by weight_q and w_delta
print(weight_int) # you should see clean integer numbers, it should be extremely close to integers

clipping threshold weight alpha: 2.087000, activation alpha: 3.201000
tensor([[[[ 2.9995, -3.9993,  1.9996],
          [-0.9998, -5.9989,  3.9993],
          [-6.9987, -3.9993, -2.9995]],

         [[ 0.0000,  0.0000,  1.9996],
          [-1.9996, -1.9996, -0.0000],
          [-1.9996, -3.9993, -0.9998]],

         [[ 0.0000, -0.9998,  0.9998],
          [ 0.0000, -3.9993, -0.9998],
          [ 1.9996, -2.9995, -1.9996]],

         ...,

         [[ 2.9995,  0.0000, -0.0000],
          [-0.0000, -3.9993, -2.9995],
          [ 0.9998, -2.9995, -2.9995]],

         [[ 1.9996,  0.0000,  1.9996],
          [ 0.9998, -0.9998, -3.9993],
          [ 2.9995,  4.9991,  0.0000]],

         [[ 1.9996, -2.9995,  0.9998],
          [ 5.9989,  4.9991,  3.9993],
          [-5.9989, -1.9996,  0.0000]]],


        [[[ 0.9998, -0.9998,  0.0000],
          [ 0.9998, -1.9996, -0.0000],
          [-0.9998, -0.9998, -0.9998]],

         [[-0.0000,  0.9998,  0.9998],
          [ 0.9998,  0.9998,  1.9996],
  

In [94]:
x_bit = 4
# input of the 2nd conv layer
saved_entry = save_output.outputs[2]
module_name, module_in = saved_entry
print ("Module name for saved input:", module_name)  
x = module_in[0].detach().clone()
x_alpha  = 3.201000
x_delta = x_alpha / ((2 ** x_bit)-1)

act_quant_fn = act_quantization(x_bit) # define the quantization function
x_q = act_quant_fn(x, x_alpha)         # create the quantized value for x

x_int = x_q / x_delta  # x_int can be calculated by x_q and x_delta
print(x_int) # you should see clean integer numbers 


Module name for saved input: layer1.0.conv1
tensor([[[[ 3.0000,  6.0000,  7.0000,  ...,  8.0000,  8.0000,  6.0000],
          [ 4.0000,  7.0000,  8.0000,  ...,  9.0000,  8.0000, 12.0000],
          [ 3.0000,  7.0000,  8.0000,  ...,  9.0000,  9.0000, 11.0000],
          ...,
          [13.0000,  6.0000,  5.0000,  ...,  7.0000,  3.0000,  0.0000],
          [ 9.0000,  7.0000,  5.0000,  ...,  2.0000, 11.0000,  0.0000],
          [ 5.0000,  5.0000,  4.0000,  ...,  2.0000,  5.0000,  0.0000]],

         [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  1.0000,  1.0000,  ...,  1.0000,  2.0000,  2.0000]],

         [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.00

In [95]:
conv_int = torch.nn.Conv2d(in_channels = 64, out_channels=64, kernel_size = 3, bias = False)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)

output_int = conv_int(x_int) # output_int can be calculated with conv_int and x_int
output_recovered = output_int * x_delta * w_delta  # recover with x_delta and w_delta
print(output_recovered)

tensor([[[[-1.1450e+00, -1.1450e+01, -9.7327e+00,  ..., -4.1984e+00,
           -6.1068e+00, -1.8320e+01],
          [-1.5903e+00, -1.2913e+01, -9.9235e+00,  ..., -2.6717e+00,
           -3.6259e+00, -1.4376e+01],
          [-3.4987e+00, -1.2086e+01, -6.3612e+00,  ..., -6.9973e-01,
            3.8167e-01, -1.1450e+01],
          ...,
          [-2.9134e+01, -2.6844e+01, -2.6399e+01,  ..., -2.2773e+01,
           -3.2697e+01, -2.9516e+01],
          [-2.2773e+01, -2.2328e+01, -2.5063e+01,  ..., -1.7875e+01,
           -2.7290e+01, -2.3155e+01],
          [-2.1819e+01, -1.5012e+01, -1.9211e+01,  ..., -1.5203e+01,
           -1.3295e+01, -2.4045e+01]],

         [[ 5.4070e+01,  5.2798e+01,  5.3371e+01,  ...,  4.8345e+01,
            4.5992e+01,  4.1666e+01],
          [ 4.9490e+01,  4.8600e+01,  5.0063e+01,  ...,  4.3574e+01,
            4.1793e+01,  3.8358e+01],
          [ 4.6437e+01,  4.6500e+01,  4.8472e+01,  ...,  4.4719e+01,
            4.3129e+01,  3.8803e+01],
          ...,
     

In [98]:
#### input floating number / weight quantized version

conv_ref = torch.nn.Conv2d(in_channels = 64, out_channels=64, kernel_size = 3, bias = False)
conv_ref.weight = mod.weight_q

output_ref = conv_ref(x)
print(output_ref)    

tensor([[[[-1.1532e+00, -1.1383e+01, -8.9341e+00,  ..., -4.1374e+00,
           -6.3392e+00, -1.8288e+01],
          [-1.5302e+00, -1.1977e+01, -9.7487e+00,  ..., -2.4339e+00,
           -3.5266e+00, -1.4036e+01],
          [-3.7124e+00, -1.0970e+01, -6.7664e+00,  ..., -4.6223e-01,
            1.9162e-01, -9.9417e+00],
          ...,
          [-2.9828e+01, -2.7429e+01, -2.6654e+01,  ..., -2.2617e+01,
           -3.2656e+01, -3.0112e+01],
          [-2.2900e+01, -2.3543e+01, -2.4849e+01,  ..., -1.7114e+01,
           -2.7560e+01, -2.4075e+01],
          [-2.0955e+01, -1.5818e+01, -2.0526e+01,  ..., -1.5482e+01,
           -1.4689e+01, -2.4806e+01]],

         [[ 5.4460e+01,  5.2927e+01,  5.3573e+01,  ...,  4.8487e+01,
            4.6296e+01,  4.1812e+01],
          [ 4.9419e+01,  4.8506e+01,  4.9826e+01,  ...,  4.3696e+01,
            4.1687e+01,  3.8765e+01],
          [ 4.6172e+01,  4.6188e+01,  4.8053e+01,  ...,  4.4726e+01,
            4.3018e+01,  3.9337e+01],
          ...,
     

In [99]:
difference = abs( output_ref - output_recovered )
print(difference.mean())  ## It should be small, e.g.,2.3 in my trainned model

tensor(0.9827, device='cuda:0', grad_fn=<MeanBackward0>)


In [100]:
#### input floating number / weight floating number version

conv_ref = torch.nn.Conv2d(in_channels = 64, out_channels=64, kernel_size = 3, bias = False)
weight = mod.weight
mean = weight.data.mean()
std = weight.data.std()
conv_ref.weight = torch.nn.parameter.Parameter(weight.add(-mean).div(std))

output_ref = conv_ref(x)
print(output_ref)


tensor([[[[-8.4574e-01, -1.1445e+01, -9.0954e+00,  ..., -4.2734e+00,
           -6.4664e+00, -1.8211e+01],
          [-1.5285e+00, -1.2299e+01, -1.0206e+01,  ..., -2.8503e+00,
           -3.9378e+00, -1.4211e+01],
          [-3.7134e+00, -1.1298e+01, -7.2323e+00,  ..., -8.3200e-01,
           -2.3559e-01, -1.0102e+01],
          ...,
          [-2.9319e+01, -2.6865e+01, -2.6056e+01,  ..., -2.2011e+01,
           -3.1824e+01, -2.8804e+01],
          [-2.2433e+01, -2.2940e+01, -2.4392e+01,  ..., -1.6752e+01,
           -2.6902e+01, -2.3336e+01],
          [-2.0725e+01, -1.5458e+01, -2.0318e+01,  ..., -1.5555e+01,
           -1.4081e+01, -2.4665e+01]],

         [[ 5.4835e+01,  5.3816e+01,  5.4462e+01,  ...,  4.9114e+01,
            4.6829e+01,  4.1362e+01],
          [ 4.9281e+01,  4.8989e+01,  5.0309e+01,  ...,  4.3573e+01,
            4.1664e+01,  3.8076e+01],
          [ 4.6199e+01,  4.6562e+01,  4.8389e+01,  ...,  4.4428e+01,
            4.2862e+01,  3.8709e+01],
          ...,
     

In [101]:
difference = abs( output_ref - output_recovered )
print(difference.mean())  ## It should be small, e.g.,2.3 in my trainned model

tensor(1.3720, device='cuda:0', grad_fn=<MeanBackward0>)


In [4]:
# Now construct a 2 bit version model
model_name_2bit = "Resnet20_quant_2bit"
model_2bit = resnet20_quant_2bit()
model_2bit.cuda()
print(model_2bit)

ResNet_Cifar_2bit(
  (conv1): QuantConv2d(
    3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
    (weight_quant): weight_quantize_fn()
  )
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock_2bit(
      (conv1): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (conv2): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock_2bit(
      (conv1): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=Fa

In [9]:
fdir = 'result/'+str(model_name_2bit)+'/model_best.pth.tar'

checkpoint = torch.load(fdir)
model_2bit.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model_2bit.cuda()
model_2bit.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model_2bit(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 8237/10000 (82%)

